<a href="https://colab.research.google.com/github/maodo86/mon-application/blob/main/Predicteur_anomalies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from tensorflow.keras.models import load_model

data_model = load_model(r"C:\Users\dell\Desktop\Projet_Analyse_Magal2021\defaut_tension_model.h5")
data_scaler = joblib.load(r"C:\Users\dell\Desktop\Projet_Analyse_Magal2021\tension_scaler.pkl")

st.title("Prediction anomalie tension Postes de Distribution")

def collect_donnes(lien):
    #bibliothèques nécessairesCharger, prétraiter et examiner les données
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    %matplotlib inline
    #Téléchargement du jeu de données et visualisations des 5 premières lignes
    data=pd.read_excel(lien, parse_dates=['Dates'], index_col='Dates')
    data.head()
    return data

def pretraitement(data):
    #Sélection de variables 
    df=data[['OUVRAGE', 'V1', 'V2', 'V3', 'I1', 'I2', 'I3',
       'Puissance Active']]
    poste=df.OUVRAGE[0]
    #Renommer la feature Puissance Active en P
    df.rename(columns={'Puissance Active':'P'}, inplace=True)
    #Traitement des valeurs manquantes: suppression des lignes où tous les features sont null
    df.dropna(subset=['V1','V2','V3','I1','I2','I3','P'], axis=0, how='all', inplace=True)
    #Remplacement des valeurs null restantes par la moyennne: technique d'imputation par la moyenne
    df['V1'].fillna(value=df['V1'].mean(), axis=0, inplace=True)
    df['V2'].fillna(value=df['V2'].mean(), axis=0, inplace=True)
    df['V3'].fillna(value=df['V3'].mean(), axis=0, inplace=True)
    #Selection de variables importantes: Features Engeneering
    Var=['V1','V2','V3']
    data_traite=df[Var]
    return data_traite

def prediction(lien):
    from tensorflow.keras.models import load_model
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from smtplib import SMTP
    
   
    data = collect_donnes(lien)
    data_traite = pretraitement(data)
    scaled_data = scaler.transform(data_traite)
    
    scaled_data_reshap=scaled_data.reshape(scaled_data.shape[0],1,scaled_data.shape[1])
 
    
    
    X_pred = model.predict(scaled_data_reshap)
    X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
    X_pred = pd.DataFrame(X_pred, columns=train.columns)
    X_pred.index = data_traite.index

    scored = pd.DataFrame(index=data_traite.index)
    Xtrain = scaled_data.reshape(scaled_data_reshap.shape[0], scaled_data_reshap.shape[2])
    scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtrain), axis = 1)
    #calculer le seuil de décision
    std_x_train=np.std(scored.Loss_mae)*3
    mean_x_train=np.mean(scored.Loss_mae)
    seuil =std_x_train
    
    #calcul des pertes dans le jeu de donnees

    scored['seuil']=seuil
    scored['Anomalie']=scored['Loss_mae']>scored['seuil']
    #Visualisation du résultat
    #scored.plot(logy=True, figsize=(16,9), color=['blue', 'red'])
    anomalies= scored[scored.Anomalie==True]
    anomalies_nbr=anomalies.resample('D').mean()
    anomalies_detectees = anomalies_nbr.dropna()
    anomalies_detectees = anomalies_detectees.drop(['Loss_mae', 'seuil'], axis=1)
    
    #Notification par e-mail
    # SMTP_SSL Example
    server_ssl = smtplib.SMTP_SSL("smtp.gmail.com", 465)
    server_ssl.ehlo() # optional, called by login()
    server_ssl.login('senemao86@gmail.com', '222205058686')  
    # ssl server doesn't support or need tls, so don't call server_ssl.starttls() 
    msg= str(anomalies_detectees)        # ['Anomalie'])
    server_ssl.sendmail('senemao86@gmail.com', ['diopkhals@gmail.com', 'senemao86@gmail.com'], msg )
    #server_ssl.quit()
    server_ssl.close()
    print("Historique des anomalies au poste {}:".format(poste)) 

    return anomalies_detectees
#Definir notre page web
def main():       
    # front end elements of the web page 
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Streamlit Prediction Anomalie Tension ML App</h1> 
    </div> 
    """
      
    # display the front end aspect
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # following lines create boxes in which user can enter data required to make prediction 
    lien = st.file_uploader('Fichier des donnees',["xlsx"])

    result =""
      
    # when 'Predict' is clicked, make the prediction and store it 
    if st.button("Predict"): 
        result = prediction(lien)
        st.success('Votre prediction est {}'.format(result))
     
    if __name__=='__main__': 
        main()

